# Analyzing the behavior before selecting pictures

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
day = '0226'

In [ ]:

headers = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10']
data_f = pd.read_csv(''+day+'/final_data.csv', sep=',', skiprows = 1, header=None, names=headers, dtype={"col1": int, "col2": float, "col3": object, "col4": object, "col5": float, "col6": float, "col7": object, "col8": object, "col9": object, "col10": object})
data = data_f.to_numpy()
print(data)

## output format
- trial_index, time_start, time_end, duration(response time), box_chosen, correct_or_not, time_on_selected_picture, time_on_pic_0.2, time_on_pic_0.4, time_on_pic_0.6, time_on_pic_0.8, percentage_on_pic_0.2, percentage_on_pic_0.4, percentage_on_pic_0.6, percentage_on_pic_0.8
- time: 1s = 2000 lines, 0.2s = 400, 0.4s = 800, 0.6s = 1200, 0.8s = 1600. Thus, 1 line = 1/2000s

In [ ]:
output = np.empty((100, 16), dtype=object)
output[:] = ""


In [ ]:
df_header = pd.read_csv(''+day+'/J_'+day+'_BHV/2023'+day+'_bhv.csv')
print(''+day+'/J_'+day+'_BHV/2023'+day+'_bhv.csv')
print(df_header)

In [ ]:
print(df_header['pic_chosen'])
print(df_header['pic_chosen_position'].to_numpy().shape)

In [ ]:
def in_08(i, end_data_index):
    if (end_data_index - i) / 2000 <= 0.8:
        return True
    return False


def in_06(i, end_data_index):
    if (end_data_index - i) / 2000 <= 0.6:
        return True
    return False


def in_04(i, end_data_index):
    if (end_data_index - i) / 2000 <= 0.4:
        return True
    return False


def in_02(i, end_data_index):
    if (end_data_index - i) / 2000 <= 0.2:
        return True
    return False


def cal_response_time(start_data_index, end_data_index, data):
    response_time = (data[end_data_index - 2000][0] - data[start_data_index][0]) / 2000
    return response_time


def cal_time_on_pic(chosen_pic, start_data_index, end_data_index, data):
    num_chosen_pic = 0
    num_chosen_pic_08 = 0
    num_chosen_pic_06 = 0
    num_chosen_pic_04 = 0
    num_chosen_pic_02 = 0
    num_pic = 0
    num_pic_08 = 0
    num_pic_06 = 0
    num_pic_04 = 0
    num_pic_02 = 0
    for i in range(start_data_index, end_data_index):
        if '.png' in data[i][-1]:
            num_pic += 1
            if in_08(i, end_data_index):
                num_pic_08 += 1
            if in_06(i, end_data_index):
                num_pic_06 += 1
            if in_04(i, end_data_index):
                num_pic_04 += 1
            if in_02(i, end_data_index):
                num_pic_02 += 1

            if data[i][-1] == chosen_pic:
                num_chosen_pic += 1
                if in_08(i, end_data_index):
                    num_chosen_pic_08 += 1
                if in_06(i, end_data_index):
                    num_chosen_pic_06 += 1
                if in_04(i, end_data_index):
                    num_chosen_pic_04 += 1
                if in_02(i, end_data_index):
                    num_chosen_pic_02 += 1
    
    time_on_pic = num_chosen_pic / 2000
    time_on_pic_08 = num_chosen_pic_08 / 2000
    time_on_pic_06 = num_chosen_pic_06 / 2000
    time_on_pic_04 = num_chosen_pic_04 / 2000
    time_on_pic_02 = num_chosen_pic_02 / 2000
    segment_num = [num_chosen_pic, num_chosen_pic_02, num_chosen_pic_04, num_chosen_pic_06, num_chosen_pic_08]
    total_view_num = [num_pic, num_pic_02, num_pic_04, num_pic_06, num_pic_08]
    prop, prop_02, prop_04, prop_06, prop_08 = cal_prop_on_pic(segment_num, total_view_num)
    return time_on_pic, time_on_pic_02, time_on_pic_04, time_on_pic_06, time_on_pic_08, prop, prop_02, prop_04, prop_06, prop_08


def cal_prop_on_pic(segment_num, total_view_num): #segment_time (5,)
    prop =  0 if (total_view_num[0] == 0) else (segment_num[0] / total_view_num[0])
    prop_02 = 0 if (total_view_num[1] == 0) else (segment_num[1] / total_view_num[1])
    prop_04 = 0 if (total_view_num[2] == 0) else (segment_num[2] / total_view_num[2])
    prop_06 = 0 if (total_view_num[3] == 0) else (segment_num[3] / total_view_num[3])
    prop_08 = 0 if (total_view_num[4] == 0) else (segment_num[4] / total_view_num[4])
    return prop, prop_02, prop_04, prop_06, prop_08 


In [ ]:
trial_index = 0
current_data_index = 0
# 0:5 trial_index, time_start, time_end, duration(response time), box_chosen, correct_or_not, 
# 6:10 time_on_selected_picture, time_on_pic_0.2, time_on_pic_0.4, time_on_pic_0.6, time_on_pic_0.8, 
# 11:15 percent_on_pic_trial, percentage_on_pic_0.2, percentage_on_pic_0.4, percentage_on_pic_0.6, percentage_on_pic_0.8
print(np.shape(data)[0])
try:
    while current_data_index < np.shape(data)[0] and trial_index < 100:
        if current_data_index % 100000 == 0:
            print("track", current_data_index)
        if data[current_data_index][-2] is np.nan:
            current_data_index += 1
        elif 'forcedchoice_start' in data[current_data_index][-2]:
            output[trial_index][0] = trial_index
            output[trial_index][1] = data[current_data_index][1]
            start_data_index = current_data_index
            while not data[current_data_index][-2] is np.nan and 'forcedchoice_start' in data[current_data_index][-2]:
                current_data_index += 1
            end_data_index = current_data_index
            print("trial end at", end_data_index)
            output[trial_index][2] = data[end_data_index - 1][1]
            output[trial_index][3] = cal_response_time(start_data_index, end_data_index, data)
            output[trial_index][4] = df_header['pic_chosen_position'][trial_index]
            output[trial_index][5] = df_header['response'][trial_index]
            chosen_pic = df_header['pic_chosen'][trial_index]
            output[trial_index][6], output[trial_index][7], output[trial_index][8], output[trial_index][9], output[trial_index][10], output[trial_index][11], output[trial_index][12], output[trial_index][13], output[trial_index][14], output[trial_index][15]  = cal_time_on_pic(chosen_pic, start_data_index, end_data_index - 2000, data)
            trial_index += 1
        elif 'forcedmemory_start' in data[current_data_index][-2]:
            current_data_index += 1
        else:
            print("error occurs in line", current_data_index)
            raise(ValueError) 
except ValueError:
    print("???")

In [ ]:
print(output)

In [ ]:
# 0:5 trial_index, time_start, time_end, duration(response time), box_chosen, correct_or_not, 
# 6:10 time_on_selected_picture, time_on_pic_0.2, time_on_pic_0.4, time_on_pic_0.6, time_on_pic_0.8, 
# 11:15 percent_on_pic_trial, percentage_on_pic_0.2, percentage_on_pic_0.4, percentage_on_pic_0.6, percentage_on_pic_0.8
correct_response_time = []
incorrect_response_time = []
target_valid = []
target_valid_08 = []
target_valid_06 = []
target_valid_04 = []
target_valid_02 = []
foil_valid = []
foil_valid_08 = []
foil_valid_06 = []
foil_valid_04 = []
foil_valid_02 = []

for record in output:
    if record[5] == 1:
        correct_response_time.append(record[3])
        if record[11] != 0:
            target_valid.append(record[11])
        if record[3] >= 0.2 and record[12] != 0:
            target_valid_02.append(record[12])
        if record[3] >= 0.4 and record[13] != 0:
            target_valid_04.append(record[13])
        if record[3] >= 0.6 and record[14] != 0:
            target_valid_06.append(record[14])
        if record[3] >= 0.8 and record[15] != 0:
            target_valid_08.append(record[15])
    elif record[5] == 0:
        incorrect_response_time.append(record[3])
        if record[11] != 0:
            foil_valid.append(record[11])
        if record[3] >= 0.2 and record[12] != 0:
            foil_valid_02.append(record[12])
        if record[3] >= 0.4 and record[13] != 0:
            foil_valid_04.append(record[13])
        if record[3] >= 0.6 and record[14] != 0:
            foil_valid_06.append(record[14])
        if record[3] >= 0.8 and record[15] != 0:
            foil_valid_08.append(record[15])



In [ ]:
foil_valid

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title('Percentage Viewing Time_all trials')
#plt.bar(['target_viewing_time_percent','foil_viewing_time_percent'], [df_all['target_viewing_time_percent'].mean(),df_all['foil_viewing_time_percent'].mean()],color=['blue','yellow'], width=0.)
plt.bar(['Target','Foil'], [np.mean(target_valid), np.mean(foil_valid)],color=['blue','yellow'], width=0.8)

plt.subplot(1,2,2)
plt.title('time_before_response')
x_axis = ["0.8", "0.6", "0.4", "0.2"]
mean_values_target = [np.mean(target_valid_08), np.mean(target_valid_06), np.mean(target_valid_04), np.mean(target_valid_02)]
mean_values_foil = [np.mean(foil_valid_08), np.mean(foil_valid_06), np.mean(foil_valid_04), np.mean(foil_valid_02)]
print(foil_valid_08)
plt.plot(x_axis, mean_values_target, marker='o', label='target')
# Plot the second line
plt.plot(x_axis, mean_values_foil, marker='o', label='foil')

plt.legend()